<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.


## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle


# Изучим данные
data = pd.read_csv('/datasets/Churn.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [2]:
# Преобразуем категориальные признаки в количественные. Удалим столбец RowNumber. 
data_ohe = pd.get_dummies(data, columns=['Geography', 'Gender'], drop_first=True)
data_ohe = data_ohe.drop(['RowNumber', 'Surname', 'CustomerId'], axis=1)

In [44]:
# Удалим дубликаты. Также удалим те строки, где содержится значение NaN.
data_ohe = data_ohe.fillna(random.randrange(0, 10.0, 1))
data_ohe = data_ohe.drop_duplicates()

In [63]:
data_ohe['Tenure'].unique()

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0.])

In [62]:
# Посмотрим новую таблицу
data_ohe

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7.0,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3.0,75075.31,2,1,0,92888.52,1,1,0,1


In [46]:
# Выделим целевой признак
target = data_ohe['Exited']
features = data_ohe.drop('Exited', axis=1)
# Разделим выборку на тренировочную и валидационную
target = data_ohe['Exited']
features = data_ohe.drop('Exited', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.5, 
                                                                              random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5, 
                                                                              random_state=12345)

Вывод:
- При изучении данных пропуски были обнаружены лишь в одном столбце - 'Tenure';
- Дубликатов в данных не обнгаружены;
- Удалили столбец 'RowNumber', так как он полность копирует индексы строк;
- Удалили столбец 'Surname' за ненадобностью: клиентов идентифицирует их 'CustomerId';
- Заполнили пропуски в столбце 'Tenure' случайными значениями;
- Преобразовали категориальные признаки в количественные - это столбцы 'Geography' и 'Gender';
- Во избежании дамми-ловушки в аргументе drop_first функции pd.get_dummies указали значение True.

## Исследование задачи

In [47]:
# Исследуем баланс классов
class_1 = data_ohe['Exited'].value_counts()[1]
class_0 = data_ohe['Exited'].value_counts()[0]
print("Количество людей, ушедших из банка:", class_1)
print("Количество людей, оставшихся в банке:", class_0)
N = class_0 / class_1
print("Вес класса 1 - люди, ушедшие из банка", N)

Количество людей, ушедших из банка: 2037
Количество людей, оставшихся в банке: 7963
Вес класса 1 - люди, ушедшие из банка 3.9091801669121256


In [48]:
# Обучим модель логистическрй регрессии при дисбалансе классов
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight=None)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print(f1_score(target_valid, predicted_valid))

0.11647254575707155


In [49]:
# Обучим модель решающего дерева с разной глубиной при дисбалансе классов
best_f1_score = 0
best_depth = 0
for depth in range(1, 15, 1):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight=None)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    if f1 > best_f1_score:
        best_f1_score = f1
        best_depth = depth
    else:
        best_f1_score += 0
print(best_f1_score)
print(best_depth)

0.5320665083135392
7


Вывод:
- Исследовали баланс классов. Количество людей, которые остались в банке, практически в 4 раза больше чем людей, которые ушли из банка;
- Обучили модель логистическрй регрессии при дисбалансе классов. Значение метрики F1 в таком случае равно 0.0

## Борьба с дисбалансом

In [50]:
# Обучим модель логистическрй регрессии при взвешивании классов
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("Метрика F1 равна:", f1_score(target_valid, predicted_valid))

Метрика F1 равна: 0.4323284156510584


In [51]:
# Обучим модель решающего дерева с разной глубиной при взвешивании классов
best_f1_score = 0
best_depth = 0
for depth in range(1, 20, 1):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight='balanced')
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    if f1 > best_f1_score:
        best_f1_score = f1
        best_depth = depth
    else:
        best_f1_score += 0
print("Метрика F1 равна:", best_f1_score)
print("Лучшая глубина:", best_depth)

Метрика F1 равна: 0.5645295587010825
Лучшая глубина: 6


In [52]:
%%time

best_f1_score_2 = 0
best_depth_2 = 0
best_n = 0

for depth in range(1, 25, 1):
    for n in range(1, 60, 1):
        model = RandomForestClassifier(n_estimators=n, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1_score_2:
            best_f1_score_2 = f1
            best_depth_2 = depth
            best_n = n
        else:
            best_f1_score_2 += 0
print("Метрика F1 равна:", best_f1_score_2)
print("Лучшая глубина:", best_depth)
print("Лучшее количество деревьев: ", best_n)

Метрика F1 равна: 0.5738498789346246
Лучшая глубина: 6
Лучшее количество деревьев:  57
CPU times: user 4min 2s, sys: 863 ms, total: 4min 3s
Wall time: 4min 3s


In [53]:
# Сделаем upsampling. Проверим метрику F1 на модели логситической регрессии.
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)
model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.43154574132492113


In [54]:
# Сделали dawnsampling.
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [55]:
%%time
# Обучили модель на сбалансированных данных.
best_f1_score_2 = 0
best_depth_2 = 0
best_n = 0
best_model = None
for depth in range(1, 25, 1):
    for n in range(1, 60, 1):
        model = RandomForestClassifier(n_estimators=n, max_depth=depth, random_state=12345)
        model.fit(features_downsampled, target_downsampled)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1_score_2:
            best_f1_score_2 = f1
            best_depth_2 = depth
            best_n = n
            best_model = model
        else:
            best_f1_score_2 += 0
print(best_f1_score_2)
print("Лучшая глубина:", best_depth)
print("Лучшее количество деревьев:", best_n)

0.5647425897035881
Лучшая глубина: 6
Лучшее количество деревьев: 26
CPU times: user 2min 18s, sys: 764 ms, total: 2min 19s
Wall time: 2min 19s


In [56]:
%%time
# Обучили модель на сбалансированных данных.
best_f1_score_2 = 0
best_depth_2 = 0
best_n = 0
best_model_2 = None
for depth in range(1, 25, 1):
    for n in range(1, 60, 1):
        model = RandomForestClassifier(n_estimators=n, max_depth=depth, random_state=12345)
        model.fit(features_upsampled, target_upsampled)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1_score_2:
            best_f1_score_2 = f1
            best_depth_2 = depth
            best_n = n
            best_model_2 = model
        else:
            best_f1_score_2 += 0
print(best_f1_score_2)
print("Лучшая глубина:", best_depth)
print("Лучшее количество деревьев:", best_n)

0.5976520811099253
Лучшая глубина: 6
Лучшее количество деревьев: 57
CPU times: user 5min 14s, sys: 1.47 s, total: 5min 16s
Wall time: 5min 16s


Вывод:
 - Обучили модель логистической регрессии при взвешивании классов. Результат метрики F1: 0.4323284156510584. Этого недостаточно для поставленной в цели исследования задачи.
 - Потом обучили модель решающего дерева при взвешивании классов, изменяя параметр max_depth. Метрика F1 стала равна  0.5645295587010825. Глубина дерева при наилучшем показателе F1 равна 6.
 - Обучили модель случайного леса. Нужного значения метрики (0.59) получено не было.
 - Также были применены техники downsampling и upsampling.
 - Лучшее значение метрики F1 - 0.5976520811099253.

## Тестирование модели

In [57]:
# Проводим тестирование
predicted_test = best_model_2.predict(features_test)
print("Значение метрики F1:", f1_score(target_test, predicted_test))

Значение метрики F1: 0.6087866108786611


In [58]:
# Исследуем метрику AUC-ROC
auc_roc = roc_auc_score(target_test, predicted_test)
print(auc_roc)

0.7453771149901259


Вывод:
Финальное тестирование проведено. Проверили метрику F1 - 0.6087866108786611. Метрика AUC-ROC - 0.7453771149901259